In [13]:
import pandas as pd
import os
from datetime import timedelta
import numpy as np # Diperlukan untuk float('inf') pada bins

# --- KONFIGURASI ---
NAMA_FILE_CSV_INPUT = "18_05_2025-24_05_2025.csv" # GANTI JIKA NAMA FILE CSV INPUT ANDA BERBEDA
NAMA_FILE_CSV_OUTPUT = "hasil_analisis_trade_lengkap.csv" # Nama file output untuk data trade lengkap

# --- FUNGSI UNTUK IDENTIFIKASI KOLOM ---
def identifikasi_nama_kolom(daftar_alias, peta_kolom_aktual):
    for alias in daftar_alias:
        if alias in peta_kolom_aktual:
            return peta_kolom_aktual[alias]
    return None

# --- FUNGSI UNTUK MENGHITUNG STREAK ---
def hitung_streaks(seri_profit):
    max_win_streak = 0
    current_win_streak = 0
    max_loss_streak = 0
    current_loss_streak = 0
    for profit in seri_profit:
        if profit > 0:
            current_win_streak += 1
            current_loss_streak = 0
        elif profit < 0:
            current_loss_streak += 1
            current_win_streak = 0
        else:
            current_win_streak = 0
            current_loss_streak = 0
        if current_win_streak > max_win_streak: max_win_streak = current_win_streak
        if current_loss_streak > max_loss_streak: max_loss_streak = current_loss_streak
    return max_win_streak, max_loss_streak

# --- FUNGSI UTAMA ANALISIS ---
def analisis_performa_trading(nama_file_input, nama_file_output):
    try:
        # --- 1. Memuat Data ---
        if not os.path.exists(nama_file_input):
            print(f"ERROR: File input '{nama_file_input}' tidak ditemukan.")
            return

        df = pd.read_csv(nama_file_input)
        print(f"Berhasil memuat file: {nama_file_input}")
        print(f"Jumlah baris data awal: {len(df)}")
        # Menghilangkan print berikut:
        # print(f"Nama kolom aktual di CSV Anda (setelah dimuat Pandas): {df.columns.tolist()}")

        # --- 2. Identifikasi Kolom ---
        peta_kolom_aktual = {
            col.lower().replace(' ', '').replace('_', '').replace('/', '').replace('.', ''): col
            for col in df.columns
        }
        # Menghilangkan print berikut:
        # print(f"Peta kolom yang diproses untuk pencarian (kunci:nilai asli): {peta_kolom_aktual}")

        alias_simbol = ['symbol', 'pair', 'item', 'instrument']
        alias_tipe = ['type', 'direction', 'action', 'dealaction']
        alias_harga_buka = ['openingprice', 'openprice', 'price', 'entryprice']
        alias_harga_tutup = ['closingprice', 'closeprice', 'exitprice']
        alias_profit = ['profitusc', 'profit', 'pl', 'netprofit', 'pnl']
        alias_waktu_buka = ['openingtimeutc', 'opentime', 'open_time', 'time', 'entrytime']
        alias_waktu_tutup = ['closingtimeutc', 'closetime', 'close_time', 'exittime']
        alias_close_reason = ['closereason', 'close_reason', 'reason']

        kolom_simbol = identifikasi_nama_kolom(alias_simbol, peta_kolom_aktual)
        kolom_tipe = identifikasi_nama_kolom(alias_tipe, peta_kolom_aktual)
        kolom_harga_buka = identifikasi_nama_kolom(alias_harga_buka, peta_kolom_aktual)
        kolom_harga_tutup = identifikasi_nama_kolom(alias_harga_tutup, peta_kolom_aktual)
        kolom_profit = identifikasi_nama_kolom(alias_profit, peta_kolom_aktual)
        kolom_waktu_buka = identifikasi_nama_kolom(alias_waktu_buka, peta_kolom_aktual)
        kolom_waktu_tutup = identifikasi_nama_kolom(alias_waktu_tutup, peta_kolom_aktual)
        kolom_close_reason = identifikasi_nama_kolom(alias_close_reason, peta_kolom_aktual)

        # Menghilangkan blok print berikut:
        # print("\n--- Hasil Identifikasi Kolom (Otomatis) ---")
        # print(f"Kolom Simbol         : '{kolom_simbol}'")
        # print(f"Kolom Tipe Trade     : '{kolom_tipe}'")
        # print(f"Kolom Harga Buka     : '{kolom_harga_buka}'")
        # print(f"Kolom Harga Tutup    : '{kolom_harga_tutup}'")
        # print(f"Kolom Profit         : '{kolom_profit}'")
        # print(f"Kolom Waktu Buka     : '{kolom_waktu_buka}'")
        # print(f"Kolom Waktu Tutup    : '{kolom_waktu_tutup}'")
        # print(f"Kolom Alasan Tutup   : '{kolom_close_reason}'")

        kolom_penting_dasar = {"Simbol": kolom_simbol, "Tipe Trade": kolom_tipe, "Harga Buka": kolom_harga_buka, "Harga Tutup": kolom_harga_tutup, "Profit": kolom_profit}
        kolom_tidak_ditemukan_dasar = [nama for nama, kolom in kolom_penting_dasar.items() if kolom is None]
        if kolom_tidak_ditemukan_dasar:
            print("\nERROR: Kolom dasar tidak teridentifikasi. Silakan periksa nama kolom di CSV Anda dan sesuaikan alias di skrip.")
            # Menampilkan detail kolom yang tidak ditemukan bisa membantu user jika ini terjadi
            for nama in kolom_tidak_ditemukan_dasar: print(f"- Kolom '{nama}' tidak ditemukan.")
            return

        hitung_holding_period = kolom_waktu_buka and kolom_waktu_tutup
        hitung_intervention_rate = bool(kolom_close_reason)
        
        if not hitung_holding_period:
            print("\nPERINGATAN: Kolom Waktu Buka dan/atau Waktu Tutup tidak ditemukan. Rata-rata Holding Period tidak akan dihitung.")
        if not hitung_intervention_rate:
            print("\nPERINGATAN: Kolom Alasan Tutup ('close_reason') tidak ditemukan. Intervention Rate tidak akan dihitung.")

        # --- 3. Pembersihan dan Persiapan Data ---
        df[kolom_harga_buka] = pd.to_numeric(df[kolom_harga_buka], errors='coerce')
        df[kolom_harga_tutup] = pd.to_numeric(df[kolom_harga_tutup], errors='coerce')
        df[kolom_profit] = pd.to_numeric(df[kolom_profit], errors='coerce')

        kolom_untuk_dropna = [kolom_simbol, kolom_tipe, kolom_harga_buka, kolom_harga_tutup, kolom_profit]
        if hitung_holding_period:
            try:
                df[kolom_waktu_buka] = pd.to_datetime(df[kolom_waktu_buka], errors='coerce')
                df[kolom_waktu_tutup] = pd.to_datetime(df[kolom_waktu_tutup], errors='coerce')
                kolom_untuk_dropna.extend([kolom_waktu_buka, kolom_waktu_tutup])
            except Exception as e_time_parse:
                print(f"Peringatan: Gagal konversi kolom waktu: {e_time_parse}")
                hitung_holding_period = False

        df.dropna(subset=kolom_untuk_dropna, inplace=True)

        if df[kolom_tipe].dtype in ['int64', 'float64', 'int32', 'float32']:
             df['TipeTradeStandar'] = df[kolom_tipe].apply(lambda x: 'buy' if x == 0 else ('sell' if x == 1 else 'tidak_diketahui'))
        else:
            df['TipeTradeStandar'] = df[kolom_tipe].astype(str).str.lower().str.strip()

        trade_df = df[df['TipeTradeStandar'].isin(['buy', 'sell'])].copy()
        print(f"\nJumlah baris setelah pembersihan dan filter (hanya trade buy/sell): {len(trade_df)}")
        if trade_df.empty:
            print("Tidak ada data trade valid untuk dianalisis.")
            return

        # --- 4. Perhitungan Pips & Holding Period ---
        if hitung_holding_period:
            trade_df.loc[:, 'DurasiHolding'] = trade_df[kolom_waktu_tutup] - trade_df[kolom_waktu_buka]
            trade_df = trade_df[trade_df['DurasiHolding'] > pd.Timedelta(seconds=0)].copy()
            if trade_df.empty:
                # print("Tidak ada data trade dengan durasi holding valid setelah filter.") # Bisa dihilangkan jika terlalu detail
                hitung_holding_period = False
        else:
            trade_df.loc[:, 'DurasiHolding'] = pd.NaT

        daftar_pips = []
        for index, baris in trade_df.iterrows():
            simbol_val = str(baris[kolom_simbol]).upper()
            harga_buka_val = baris[kolom_harga_buka]
            harga_tutup_val = baris[kolom_harga_tutup]
            tipe_trade_val = baris['TipeTradeStandar']
            pips = 0.0
            perbedaan_harga = harga_tutup_val - harga_buka_val
            if "BTCUSD" in simbol_val:
                faktor_pips = 0.1
                pips = (perbedaan_harga * faktor_pips) if tipe_trade_val == 'buy' else (-perbedaan_harga * faktor_pips)
            elif "XAUUSD" in simbol_val:
                faktor_pips = 10.0
                pips = (perbedaan_harga * faktor_pips) if tipe_trade_val == 'buy' else (-perbedaan_harga * faktor_pips)
            daftar_pips.append(pips)

        trade_df.loc[:, 'Pips'] = daftar_pips

        trade_df.loc[:, 'HoldingRange'] = "N/A"
        if hitung_holding_period and 'DurasiHolding' in trade_df.columns:
            valid_durations = trade_df['DurasiHolding'].dropna()
            if not valid_durations.empty:
                holding_periods_minutes = valid_durations.dt.total_seconds() / 60.0
                if not holding_periods_minutes.empty:
                    bins_minutes = [-1, 1, 5, 15, 30, 60, 120, 240, 480, 1440, 2880, np.inf]
                    labels_minutes = ['<1 min', '1-5 min', '5-15 min', '15-30 min', '30-60 min',
                                      '1-2 jam', '2-4 jam', '4-8 jam', '8-24 jam', '1-2 hari', '>2 hari']
                    binned_data = pd.cut(holding_periods_minutes, bins=bins_minutes, labels=labels_minutes, right=False)
                    trade_df.loc[binned_data.index, 'HoldingRange'] = binned_data

        # --- 5. Kalkulasi Metrik Performa ---
        jumlah_total_trade = len(trade_df)
        if jumlah_total_trade == 0:
            print("Tidak ada trade yang tersisa setelah filter untuk dianalisis.")
            return

        jumlah_trade_profit = trade_df[trade_df[kolom_profit] > 0].shape[0]
        jumlah_trade_loss = trade_df[trade_df[kolom_profit] < 0].shape[0]
        total_pips = trade_df['Pips'].sum()
        total_profit_val = trade_df[kolom_profit].sum()
        win_rate = (jumlah_trade_profit / jumlah_total_trade) * 100
        max_consecutive_wins, max_consecutive_losses = hitung_streaks(trade_df[kolom_profit])

        avg_holding_period_str = "N/A"
        median_holding_period_str = "N/A"
        most_common_holding_range_str = "N/A"

        if hitung_holding_period and 'DurasiHolding' in trade_df.columns:
            valid_holding_periods = trade_df['DurasiHolding'].dropna()
            if not valid_holding_periods.empty:
                avg_hp = valid_holding_periods.mean()
                total_seconds_avg = avg_hp.total_seconds()
                d_avg, h_avg, m_avg, s_avg = int(total_seconds_avg // 86400), int((total_seconds_avg % 86400) // 3600), int((total_seconds_avg % 3600) // 60), int(total_seconds_avg % 60)
                avg_holding_period_str = f"{d_avg} hari, {h_avg} jam, {m_avg} menit, {s_avg} detik"

                median_hp = valid_holding_periods.median()
                total_seconds_med = median_hp.total_seconds()
                d_med, h_med, m_med, s_med = int(total_seconds_med // 86400), int((total_seconds_med % 86400) // 3600), int((total_seconds_med % 3600) // 60), int(total_seconds_med % 60)
                median_holding_period_str = f"{d_med} hari, {h_med} jam, {m_med} menit, {s_med} detik"
                
                if 'HoldingRange' in trade_df.columns:
                    mode_series = trade_df['HoldingRange'].dropna().mode()
                    if not mode_series.empty:
                        most_common_holding_range_str = mode_series[0]
                    else:
                        most_common_holding_range_str = "N/A (Tidak ada mode ditemukan)"
                else:
                    most_common_holding_range_str = "N/A (Kolom HoldingRange tidak dibuat)"
            else:
                avg_holding_period_str = median_holding_period_str = most_common_holding_range_str = "N/A (Data waktu tidak valid)"

        intervention_rate_str = "N/A"
        if hitung_intervention_rate and kolom_close_reason in trade_df.columns:
            valid_close_reasons = trade_df[kolom_close_reason].dropna()
            if not valid_close_reasons.empty:
                intervensi_pengguna = valid_close_reasons.astype(str).str.lower().str.strip() == 'user'
                jumlah_intervensi = intervensi_pengguna.sum()
                intervention_rate = (jumlah_intervensi / jumlah_total_trade) * 100
                intervention_rate_str = f"{intervention_rate:.2f}% ({jumlah_intervensi} dari {jumlah_total_trade} trade)"
            else: intervention_rate_str = "N/A (Data alasan tutup tidak valid)"

        # --- 6. Tampilkan Hasil Ringkasan ---
        print("\n--- HASIL ANALISIS PERFORMA TRADING ---")
        print(f"Total Jumlah Trade         : {jumlah_total_trade}")
        print(f"Jumlah Trade yang Profit   : {jumlah_trade_profit}")
        print(f"Jumlah Trade yang Loss     : {jumlah_trade_loss}")
        print(f"Win Rate                   : {win_rate:.2f}%")
        print(f"Total Pips yang Didapatkan : {total_pips:.2f} pips")
        print(f"Total Profit (kolom '{kolom_profit}') : {total_profit_val:.2f}")
        print(f"Max Consecutive Wins       : {max_consecutive_wins} trade")
        print(f"Max Consecutive Losses     : {max_consecutive_losses} trade")
        print(f"Rata-rata Holding Period   : {avg_holding_period_str}")
        print(f"Median Holding Period      : {median_holding_period_str}")
        print(f"Rentang Holding Umum       : {most_common_holding_range_str}")
        print(f"Intervention Rate (by User): {intervention_rate_str}")

        # --- 7. Simpan Trade Lengkap ke CSV Baru ---
        try:
            kolom_untuk_output = df.columns.tolist()
            kolom_tambahan_dihasilkan = ['TipeTradeStandar', 'Pips', 'DurasiHolding', 'HoldingRange']
            kolom_final_output = []
            for kol_asli in df.columns:
                if kol_asli in trade_df.columns:
                    kolom_final_output.append(kol_asli)
            for kol_baru in kolom_tambahan_dihasilkan:
                if kol_baru in trade_df.columns and kol_baru not in kolom_final_output:
                    kolom_final_output.append(kol_baru)

            trade_df.to_csv(nama_file_output, index=False, columns=kolom_final_output)
            print(f"\nTrade lengkap dengan analisis telah disimpan ke: {nama_file_output}")
        except Exception as e_save:
            print(f"ERROR saat menyimpan file output CSV: {e_save}")

        # Menghilangkan blok print berikut:
        # print("\n--- Contoh Data Trade (5 baris pertama dari file output) ---")
        # if not trade_df.empty and kolom_final_output: 
        #     kolom_valid_untuk_display = [col for col in kolom_final_output if col in trade_df.columns]
        #     if kolom_valid_untuk_display:
        #          print(trade_df[kolom_valid_untuk_display].head().to_string())
        #     else:
        #         print("Tidak ada kolom valid untuk ditampilkan dari trade_df.")
        # else:
        #     print("Tidak ada data untuk ditampilkan atau kolom output tidak valid.")

    except FileNotFoundError: print(f"ERROR: File input '{nama_file_input}' tidak ditemukan.")
    except pd.errors.EmptyDataError: print(f"ERROR: File CSV input '{nama_file_input}' kosong.")
    except KeyError as e: print(f"ERROR: Kolom '{e}' tidak ditemukan.")
    except Exception as e:
        print(f"Terjadi kesalahan yang tidak terduga: {e}")
        import traceback
        traceback.print_exc()

# --- Jalankan Analisis ---
if __name__ == "__main__":
    analisis_performa_trading(NAMA_FILE_CSV_INPUT, NAMA_FILE_CSV_OUTPUT)

Berhasil memuat file: 18_05_2025-24_05_2025.csv
Jumlah baris data awal: 60

Jumlah baris setelah pembersihan dan filter (hanya trade buy/sell): 60

--- HASIL ANALISIS PERFORMA TRADING ---
Total Jumlah Trade         : 60
Jumlah Trade yang Profit   : 28
Jumlah Trade yang Loss     : 32
Win Rate                   : 46.67%
Total Pips yang Didapatkan : 332.94 pips
Total Profit (kolom 'profit_usc') : 1836.82
Max Consecutive Wins       : 4 trade
Max Consecutive Losses     : 7 trade
Rata-rata Holding Period   : 0 hari, 1 jam, 5 menit, 32 detik
Median Holding Period      : 0 hari, 0 jam, 49 menit, 26 detik
Rentang Holding Umum       : 30-60 min
Intervention Rate (by User): 58.33% (35 dari 60 trade)

Trade lengkap dengan analisis telah disimpan ke: hasil_analisis_trade_lengkap.csv


/tmp/ipykernel_62562/2168980456.py:170: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  trade_df.loc[binned_data.index, 'HoldingRange'] = binned_data
